In [47]:

import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
%matplotlib inline
from os.path import join
import matplotlib.pyplot as plt
from billedeloader import MnistDataloader
import math
import random
from graphviz import Digraph
import pathlib
from PIL import Image
import torchvision
import time
from torch.utils.data import TensorDataset, DataLoader

In [48]:

input_path = '/Users/frederik/Desktop/torch&neural/pytorch funda/nummer1/'
training_images_filepath = join(input_path, 'mnist/train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_labels_filepath = join(input_path, 'mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
test_images_filepath = join(input_path, 'mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()
x_train = torch.tensor(x_train)
x_train_up = torch.roll(x_train, shifts=-1, dims=1)
x_train_up[:, -1, :] = 0
x_train_down = torch.roll(x_train, shifts=1, dims=1)
x_train_down[:, 0, :] = 0
x_train_left = torch.roll(x_train, shifts=-1, dims=2)
x_train_left[:, :, -1] = 0
x_train_right = torch.roll(x_train, shifts=1, dims=2)
x_train_right[:, :, 0] = 0

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = nn.Sequential(
        nn.Linear(784, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 10),
        #nn.Softmax(dim=1),              #nn.Softmax(dim=1) mse+sigmoid
)


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.CrossEntropyLoss()                  # nn.CrossEntropyLoss() nn.BCELoss() 

#prepare data
ygt = torch.tensor(y_train).double()
ygt = torch.nn.functional.one_hot(ygt.long(), num_classes=10)
#normal data
training_data = x_train*(1/255)
out = training_data.view(60000, 784).float()
#shifted data up
training_data = x_train_up*(1/255)
out_up = training_data.view(60000, 784).float()
#shifted data down
training_data = x_train_down*(1/255)
out_down = training_data.view(60000, 784).float()
#shifted data left
training_data = x_train_left*(1/255)
out_left = training_data.view(60000, 784).float()
#shifted data right
training_data = x_train_right*(1/255)
out_right = training_data.view(60000, 784).float()
#labels
labels = torch.tensor(y_train)
#all data togehter
all_data = torch.cat((out, out_up, out_down, out_left, out_right), dim=0)
all_labels = torch.cat((labels, labels, labels, labels, labels), dim=0)


In [50]:
train_all = DataLoader(dataset=TensorDataset(all_data, all_labels), batch_size=100, shuffle=True)     

In [51]:
def training_loop():
    model.train()
    for (data, target) in train_all:
        output = model(data)
        loss = loss_function(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

In [52]:

def results():
    testtotal=0
    fails=[]
    model.eval()
    for i in range(10000):


        if np.argmax(model(torch.tensor([np.concatenate(np.array(x_test[i]))*(1/255)]).float()).tolist()) == y_test[i]:
            testtotal+=1
        else:
            fails.append((i,int(np.argmax(model(torch.tensor([np.concatenate(np.array(x_test[i]))*(1/255)]).float()).tolist())), y_test[i]))

    trainingtotal=0
    #for i in range(60000):
    #    if np.argmax(model(torch.tensor([np.concatenate(np.array(x_train[i]))*(1/255)]).float()).tolist()) == np.argmax(y_train[i]):
    #        trainingtotal+=1



    print(f"{testtotal}/{len(x_test)}. i procent: {testtotal/10000*100:.02f}%")
    #print(f"{trainingtotal}/{len(x_train)}. i procent: {trainingtotal/50000*100:.02f}%")
    return testtotal, fails
        


    

In [53]:
def run(times):
    highest = 0
    for i in range(times):
        training_loop()
        testtotal, fails = results()
        if testtotal > highest:
            highest = testtotal
    print(f"best: {highest} fails: {fails}")

    #99.26

In [54]:
def egetbilleder(filename):    
    model.eval()
    image = Image.open(filename)
    imagetensor = torchvision.transforms.functional.pil_to_tensor(image)
    imagetensor = imagetensor*(1/255)
    return int(np.argmax(model(imagetensor[0].view(1, 784).float()).tolist()))

In [55]:
egetbilleder('/Users/frederik/Desktop/torch&neural/pytorch funda/nummer1/egnebilledr/pixil-frame-0-7.png')

3

In [56]:
run(5)

9796/10000. i procent: 97.96%
9848/10000. i procent: 98.48%
9855/10000. i procent: 98.55%
9872/10000. i procent: 98.72%
9868/10000. i procent: 98.68%
best: 9872 fails: [(18, 8, 3), (247, 6, 4), (326, 4, 2), (340, 3, 5), (445, 0, 6), (449, 5, 3), (495, 2, 8), (582, 2, 8), (659, 1, 2), (674, 3, 5), (684, 3, 7), (720, 8, 5), (947, 9, 8), (951, 4, 5), (965, 0, 6), (1014, 5, 6), (1033, 1, 8), (1039, 2, 7), (1112, 6, 4), (1156, 8, 7), (1178, 0, 4), (1226, 2, 7), (1232, 4, 9), (1247, 5, 9), (1299, 7, 5), (1319, 3, 8), (1393, 3, 5), (1395, 3, 2), (1530, 7, 8), (1549, 6, 4), (1621, 6, 0), (1709, 5, 9), (1754, 2, 7), (1878, 3, 8), (1901, 4, 9), (1941, 2, 7), (2053, 9, 4), (2098, 0, 2), (2109, 7, 3), (2130, 9, 4), (2135, 1, 6), (2162, 8, 5), (2224, 8, 5), (2293, 4, 9), (2387, 1, 9), (2395, 3, 8), (2454, 5, 6), (2488, 4, 2), (2534, 5, 3), (2597, 3, 5), (2607, 1, 7), (2648, 0, 9), (2654, 1, 6), (2810, 3, 5), (2877, 7, 4), (2927, 2, 3), (2939, 7, 9), (2953, 8, 3), (2995, 8, 6), (3030, 8, 6), (3060, 